# Spring 2021 Livox lidar data processing

Script to process raw .bin files from the livo lidar collected from March 2021 to June 2021
S. Filhol 2021

-------
Part 0: Convert all point cloud to daily netcdf
For each day:
1. convert to LAS
2. rotate LAS
3. extract Geotiff
4. compile daily geotiff to netcdf

-------
Part 1:  Find out when to convert bin file into DEMs

1. get wind, temperature, flux, precip and snow depth from flux station
2. identify events: precip, wind, transport.
3. create list of timestamp to convert bin to netcdf. High temporal resolution during events, low temporal resolution in low activity

-------
Part 2

1. write code to load files metadata to sort which file to process and post process
2. 


In [1]:
#from wsn_client import query
import xarray as xr
import glob
import pandas as pd
import openpylivox as opl
import pdal, json, os
import getpass
import configparser, logging, sys
sys.path.append('/home/' + getpass.getuser() + '/github/SnowyOwl/')
from appV2 import process_pcl as pp

In [2]:
# 1. Convert all point clouds to netcdf4 via geotiff

dir_bin = '/media/' + getpass.getuser() + '/My Book/SO_spring_2021_processing/bin/'
dir_netcdf = '/media/' + getpass.getuser() + '/My Book/SO_spring_2021_processing/netcdf/'
dir_daily = '/media/' + getpass.getuser() + '/My Book/SO_spring_2021_processing/daily/'

flist = glob.glob(dir_bin + '*.bin')
flist.sort()

In [3]:
meta = pd.DataFrame({'fname_archive':flist})

In [4]:
# create dataframe of file metadata
meta = pd.DataFrame({'fname_archive':flist})
#extract timestamp from filename
meta['tst'] = pd.to_datetime(meta.fname_archive.apply(lambda x: x.split('/')[-1][:-4]), format="%Y.%m.%dT%H-%M-%S")
meta['daily']=(meta.tst - pd.to_datetime('2021-03-11 00:00:00')).dt.days

In [5]:
meta.tst.diff().median()

Timedelta('0 days 00:03:00')

In [9]:
meta = meta.loc[meta.tst>='2021-04-24']

# Notes on processing

1. process one point cloud every 12 

In [6]:
# Copy bin file for the day
import shutil
import process as pp
sampling_interval=60*12


for day in meta.daily.unique()[4:6]:
    print('......')
    print('Processing Day ', day)
    daily = meta.loc[meta.daily==day]
    for file in daily.fname_archive:
        try:
            tst_data = pd.to_datetime(file.split('/')[-1][:19],format="%Y.%m.%dT%H-%M-%S")
            if (tst_data.second + 60*tst_data.minute + 3600*tst_data.hour) % sampling_interval == 0:
                shutil.copy(file, dir_daily)
        except IOerror:
            print('ERROR: cannot move ', file)
    print('-- File moved')
    pp.convert_bin_to_las(path_to_data=dir_daily)
    print('-- Bin converted to las')
    pp.tmp_rotate_point_clouds(z_range='[-20:20]', 
                               crop_corners='([-20, 10], [-5, 5])', 
                               path_to_data=dir_daily, 
                               delete_las=True)
    print('-- Rotated pcl')
    pp.extract_dem(GSD= 0.02,
                origin_x=-16.2,
                origin_y=-4,
                height=7.8,
                width=22.3,
                method='pdal',
                path_to_data=dir_daily,
                delete_las=True,
                tif_to_zip=False)
    print('-- Tif extracted')
    pp.raster_to_ds_daily(dir_daily, dir_netcdf)
    print('-- daily Netcdf compiled')
    

......
Processing Day  21
-- File moved

CONVERTING OPL BINARY DATA, PLEASE WAIT...


   : 100%|██████████| 431862/431862 [00:01<00:00, 218181.24 pts/s]


   - Point data was converted successfully to LAS, see file: /media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.01T03-00-00.bin.las

/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.01T03-00-00.bin  removed and las renamed.

CONVERTING OPL BINARY DATA, PLEASE WAIT...


   : 100%|██████████| 452655/452655 [00:02<00:00, 221648.90 pts/s]


   - Point data was converted successfully to LAS, see file: /media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.01T06-00-00.bin.las

/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.01T06-00-00.bin  removed and las renamed.

CONVERTING OPL BINARY DATA, PLEASE WAIT...


...

   - Point data was converted successfully to LAS, see file: /media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T03-00-00.bin.las

/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T03-00-00.bin  removed and las renamed.

CONVERTING OPL BINARY DATA, PLEASE WAIT...


   : 100%|██████████| 438911/438911 [00:02<00:00, 218546.21 pts/s]


   - Point data was converted successfully to LAS, see file: /media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T12-00-00.bin.las

/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T12-00-00.bin  removed and las renamed.

CONVERTING OPL BINARY DATA, PLEASE WAIT...


   : 100%|██████████| 433421/433421 [00:01<00:00, 233233.00 pts/s]


   - Point data was converted successfully to LAS, see file: /media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T15-00-00.bin.las

/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T15-00-00.bin  removed and las renamed.
-- Bin converted to las
-- Rotated pcl
-- Tif extracted
['/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T00-00-00_rot.tif', '/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T03-00-00_rot.tif', '/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T12-00-00_rot.tif', '/media/simonfi/My Book/SO_spring_2021_processing/daily/2021.04.02T15-00-00_rot.tif']
File saved:  /media/simonfi/My Book/SO_spring_2021_processing/netcdf/20210402.nc
-- daily Netcdf compiled


In [10]:
ds = xr.open_dataset('/media/arcticsnow/My Book/SO_spring_2021_processing/netcdf/20210422.nc')
ds.time


<xarray.DataArray 'time' (time: 90)>
array(['2021-04-22T00:12:00.000000000', '2021-04-22T00:24:00.000000000',
       '2021-04-22T01:12:00.000000000', '2021-04-22T01:24:00.000000000',
       '2021-04-22T01:36:00.000000000', '2021-04-22T01:48:00.000000000',
       '2021-04-22T02:00:00.000000000', '2021-04-22T02:12:00.000000000',
       '2021-04-22T02:24:00.000000000', '2021-04-22T02:36:00.000000000',
       '2021-04-22T02:48:00.000000000', '2021-04-22T03:00:00.000000000',
       '2021-04-22T03:12:00.000000000', '2021-04-22T03:24:00.000000000',
       '2021-04-22T04:12:00.000000000', '2021-04-22T04:24:00.000000000',
       '2021-04-22T04:36:00.000000000', '2021-04-22T04:48:00.000000000',
       '2021-04-22T05:00:00.000000000', '2021-04-22T05:12:00.000000000',
       '2021-04-22T05:24:00.000000000', '2021-04-22T05:36:00.000000000',
       '2021-04-22T05:48:00.000000000', '2021-04-22T06:00:00.000000000',
       '2021-04-22T06:12:00.000000000', '2021-04-22T06:24:00.000000000',
       '2021-04-22T06:36:00.000000000', '2021-04-22T06:48:00.000000000',
       '2021-04-22T07:00:00.000000000', '2021-04-22T07:12:00.000000000',
       '2021-04-22T07:24:00.000000000', '2021-04-22T07:36:00.000000000',
       '2021-04-22T07:48:00.000000000', '2021-04-22T08:00:00.000000000',
       '2021-04-22T08:12:00.000000000', '2021-04-22T08:24:00.000000000',
       '2021-04-22T09:12:00.000000000', '2021-04-22T09:24:00.000000000',
       '2021-04-22T09:36:00.000000000', '2021-04-22T09:48:00.000000000',
       '2021-04-22T10:00:00.000000000', '2021-04-22T10:12:00.000000000',
       '2021-04-22T10:24:00.000000000', '2021-04-22T10:36:00.000000000',
       '2021-04-22T10:48:00.000000000', '2021-04-22T11:00:00.000000000',
       '2021-04-22T11:12:00.000000000', '2021-04-22T11:24:00.000000000',
       '2021-04-22T12:12:00.000000000', '2021-04-22T12:24:00.000000000',
       '2021-04-22T13:12:00.000000000', '2021-04-22T13:24:00.000000000',
       '2021-04-22T13:36:00.000000000', '2021-04-22T13:48:00.000000000',
       '2021-04-22T14:00:00.000000000', '2021-04-22T14:12:00.000000000',
       '2021-04-22T14:24:00.000000000', '2021-04-22T14:36:00.000000000',
       '2021-04-22T14:48:00.000000000', '2021-04-22T15:00:00.000000000',
       '2021-04-22T15:12:00.000000000', '2021-04-22T15:24:00.000000000',
       '2021-04-22T15:36:00.000000000', '2021-04-22T15:48:00.000000000',
       '2021-04-22T16:00:00.000000000', '2021-04-22T16:12:00.000000000',
       '2021-04-22T16:24:00.000000000', '2021-04-22T17:12:00.000000000',
       '2021-04-22T17:24:00.000000000', '2021-04-22T17:36:00.000000000',
       '2021-04-22T17:48:00.000000000', '2021-04-22T18:00:00.000000000',
       '2021-04-22T18:36:00.000000000', '2021-04-22T18:48:00.000000000',
       '2021-04-22T19:00:00.000000000', '2021-04-22T19:12:00.000000000',
       '2021-04-22T19:24:00.000000000', '2021-04-22T20:36:00.000000000',
       '2021-04-22T20:48:00.000000000', '2021-04-22T21:00:00.000000000',
       '2021-04-22T21:12:00.000000000', '2021-04-22T21:24:00.000000000',
       '2021-04-22T21:36:00.000000000', '2021-04-22T21:48:00.000000000',
       '2021-04-22T22:00:00.000000000', '2021-04-22T22:36:00.000000000',
       '2021-04-22T22:48:00.000000000', '2021-04-22T23:00:00.000000000',
       '2021-04-22T23:36:00.000000000', '2021-04-22T23:48:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2021-04-22T00:12:00 ... 2021-04-22T23:48:00

In [ ]:
# Create on netcdf file per day
for day in meta.daily.unique():
    # create time variable
    time_var = xr.Variable('time', meta.tst.loc[meta.tst.dt.day==date])

    # convert by daily batches. Use functions from process_pcl!!!!
    pp.convert_bin_to_las()
    pp.rotate_point_clouds()
    pp.extract_dem()